In [2]:
import json
import pennylane as qml
import pennylane.numpy as np

In [3]:
WIRES = 2
LAYERS = 5
NUM_PARAMETERS = LAYERS * WIRES * 3

initial_params = np.random.random(NUM_PARAMETERS)

def variational_circuit(params,hamiltonian):
    """
    This is a template variational quantum circuit containing a fixed layout of gates with variable
    parameters. To be used as a QNode, it must either be wrapped with the @qml.qnode decorator or
    converted using the qml.QNode function.

    The output of this circuit is the expectation value of a Hamiltonian, somehow encoded in
    the hamiltonian argument

    Args:
        - params (np.ndarray): An array of optimizable parameters of shape (30,)
        - hamiltonian (np.ndarray): An array of real parameters encoding the Hamiltonian
        whose expectation value is returned.
    
    Returns:
        (float): The expectation value of the Hamiltonian
    """
    parameters = params.reshape((LAYERS, WIRES, 3))
    qml.templates.StronglyEntanglingLayers(parameters, wires=range(WIRES))
    return qml.expval(qml.Hermitian(hamiltonian, wires = [0,1]))

In [31]:
def optimize_circuit(params,hamiltonian):
    """Minimize the variational circuit and return its minimum value.
    You should create a device and convert the variational_circuit function 
    into an executable QNode. 
    Next, you should minimize the variational circuit using gradient-based 
    optimization to update the input params. 
    Return the optimized value of the QNode as a single floating-point number.

    Args:
        - params (np.ndarray): Input parameters to be optimized, of dimension 30
        - hamiltonian (np.ndarray): An array of real parameters encoding the Hamiltonian
        whose expectation value you should minimize.
    Returns:
        float: the value of the optimized QNode
    """
    
    # Initialize the device.
    dev = qml.device('default.qubit', wires=WIRES)
    
    @qml.qnode(dev)
    def circuit(params):
        return variational_circuit(params, hamiltonian)

    # Write your code to minimize the circuit
    opt = qml.AdamOptimizer()
    max_it = 250
    
    for _ in range(max_it):
        params, prev_cost = opt.step_and_cost(circuit, params)
        print('Cost: ', prev_cost)

    return prev_cost

In [32]:
hamiltonian = np.random.random_integers(0, 10, size=(2**WIRES, 2**WIRES))
hamiltonian = 0.5*(hamiltonian + hamiltonian.T)
initial_params = np.random.random(NUM_PARAMETERS)

dev = qml.device('default.qubit', wires=WIRES)
    
@qml.qnode(dev)
def circuit(params):
    return variational_circuit(params, hamiltonian)

opt = qml.AdamOptimizer()
params, prev_cost = opt.step_and_cost(circuit, initial_params)
print(params)
print(prev_cost)

[0.96953416 0.0230791  0.87758634 0.07416472 0.91167785 0.0517106
 0.25487882 0.4109043  0.40617063 0.52368295 0.61593515 0.60449128
 0.54745593 0.43091182 0.15296449 0.63244289 0.28800603 0.75379442
 0.13878166 0.09644219 0.16043259 0.7218639  0.31805668 0.02498401
 0.39583869 0.64431588 0.58700276 0.35267012 0.84327219 0.63196334]
12.12540883500454


In [34]:
test_input = [0.863327072347624,0.0167108057202516,0.07991447085492759,0.0854049026262154, 0.0167108057202516,0.8237963773906136,-0.07695947154193797,0.03131548733285282, 0.07991447085492759,-0.07695947154193795,0.8355417021014687,-0.11345916130631205, 0.08540490262621539,0.03131548733285283,-0.11345916130631205,0.758156886827099]
test_input = np.reshape(test_input, (4, 4))
expected_output = 0.61745341
print(optimize_circuit(initial_params, test_input))
print('Expected output: ', expected_output)


Cost:  0.8986166843025721
Cost:  0.8925937187705575
Cost:  0.8859466406010659
Cost:  0.8787037589400275
Cost:  0.8709320623044677
Cost:  0.8627099081226772
Cost:  0.8541167803510552
Cost:  0.8452394439649008
Cost:  0.8361738912902703
Cost:  0.8270210763575541
Cost:  0.8178828429634717
Cost:  0.8088573318364658
Cost:  0.8000331729304141
Cost:  0.791482522753578
Cost:  0.7832538923349459
Cost:  0.7753668696079207
Cost:  0.7678113912874592
Cost:  0.7605527471346432
Cost:  0.7535404391476567
Cost:  0.7467174983578861
Cost:  0.7400283168956956
Cost:  0.7334248208502974
Cost:  0.7268709565893174
Cost:  0.7203451256561917
Cost:  0.7138404711375744
Cost:  0.7073634759749267
Cost:  0.7009316193202422
Cost:  0.6945707578479365
Cost:  0.6883126493468911
Cost:  0.6821927945835815
Cost:  0.6762486109992978
Cost:  0.670517868557698
Cost:  0.6650372931174915
Cost:  0.6598412566327952
Cost:  0.6549605114855123
Cost:  0.650420975463591
Cost:  0.6462426217848323
Cost:  0.6424385636586808
Cost:  0.639014